In [1]:
import transformers
print(transformers.__version__)

4.26.1


In [2]:
from datasets import load_dataset
datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

Dataset wikitext downloaded and prepared to C:/Users/Admin/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 124.92it/s]


In [3]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [7]:


from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))



In [8]:
show_random_elements(datasets["train"])

,text
0,"The storyline concerning the assassination primarily follows the four conspirators who directly participate in Trujillo 's death . Antonio Imbert Barrera is one of the few conspirators who survives the violent reprisals that follow Trujillo 's assassination . Imbert is a politician who becomes disillusioned with the deception and cruelty of the Trujillo regime . His first plan to kill Trujillo was foiled by the unsuccessful attempted overthrow of the regime by Cuban paramilitary forces . Now convinced of the difficulty of his task , Imbert joins the other conspirators in plotting Trujillo 's death . Among the others is Antonio de la Maza , one of Trujillo 's personal guards . Antonio 's brother is killed as part of a government cover @-@ up and Antonio swears revenge upon Trujillo . Salvador Estrella Sadhalá , known as "" Turk "" , is a devout Catholic who , in indignation at the regime 's many crimes against God , swears an oath against Trujillo . Turk eventually turns himself in for fear that the regime was torturing his family . Both Turk and his innocent brother are then tortured for months . His father remains loyal to Trujillo and disowns Turk to his face . Despite all of this , Turk refuses to commit suicide and does not lose faith in God . He is later executed by Ramfis and other high level government men . Turk 's close friend , Amado García Guerrero , known as Amadito , is a Lieutenant in the army who gave up his beloved as proof of his loyalty to Trujillo , and then later was forced to kill her brother to prove himself to Trujillo . Amadito 's disgust with himself and disillusionment with the regime lead to his decision to help to kill Trujillo . Following the assassination he hides out with de la Maza and dies fighting . In the aftermath of the assassination , Amadito and Antonio de la Maza choose to fight the members of SIM who come to arrest them , opting to die in battle rather than be captured and tortured . \n"
1,"Hamels created and currently runs an organization known as The Hamels Foundation with his wife , Heidi Strobel . The foundation was created in 2008 . The Hamels Foundation funds childhood programs in Philadelphia and Africa . \n"
2,"Meanwhile , the 3rd Light Horse Brigade at Ballybunion was directed to move forward to Hill 70 and send one regiment to Dueidar , while the Mobile Column was ordered by GHQ to march towards Mageibra . \n"
3,
4,""" Survivor "" \n"
5,"At the end of January 2012 he was released into general prison population at State Correctional Institution – Mahanoy . He went into diabetic shock on March 30 , 2015 and has been diagnosed with active Hepatitis C. In August 2015 his attorneys filed suit in the US District Court for the Middle District of Pennsylvania upon the allegation that he has not received appropriate medical care for his health conditions . \n"
6,"The Arihant @-@ class submarines are nuclear powered ballistic missile submarines built under the Advanced Technology Vessel ( ATV ) project . They will be the first nuclear submarines designed and built by India . The submarines are 112 m ( 367 ft ) long with a beam of 11 m ( 36 ft ) , a draught of 10 m ( 33 ft ) , displacement of 6 @,@ 000 tonnes ( 5 @,@ 900 long tons ; 6 @,@ 600 short tons ) and a diving depth of 300 m ( 980 ft ) . The complement is about 95 , including officers and sailors . The boats are powered by a single seven blade propeller powered by an 83 MW ( 111 @,@ 000 hp ) pressurised water reactor and can achieve a maximum speed of 12 – 15 knots ( 22 – 28 km / h ) when surfaced and 24 knots ( 44 km / h ) when submerged . \n"
7,
8,"Aretusa was 73 @.@ 1 meters ( 239 ft 10 in ) long overall and had a beam of 8 @.@ 22 m ( 27 ft 0 in ) and an average draft of 3 @.@ 48 m ( 11 ft 5 in ) . She displaced 833 metric tons ( 820 long tons ; 918 short tons ) normally . Her propulsion system consisted of a pair of horizontal triple @-@ expansion steam engines , each driving a single screw propeller , with steam sup

In [11]:
model_checkpoint = "distilgpt2"

In [27]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\Admin/.cache\huggingface\hub\models--distilgpt2\snapshots\f241065e938b44ac52db2c5de82c8bd2fafc76d0\config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_pro

In [28]:
tokenizer

GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [29]:
def tokenize_function(examples):
    return tokenizer.tokenize(examples["text"])

In [36]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=1, remove_columns=["text"])

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]